# Building RAG Chatbots with OpenAI API, LangChain and FAISS(Facebook AI Similarity Search)



In this example, we will build a simple AI chatbot for ecommerce customer service using synthetic customer service FAQs and **R**etrieval **A**ugmented **G**eneration (RAG). We will be using OpenAI API, LangChain, and FAISS(Facebook AI Similarity Search) to build the chatbot.

By the end of the example we'll have a functioning chatbot and RAG pipeline that can hold a conversation and provide informative responses based on a knowledge base.

### Before you begin

You'll need to get an [OpenAI API key](https://platform.openai.com/account/api-keys)

* Save the key in a file in your google drive in the "Colab Notebooks" folder
* For example, name the file "openai"

## Access the key
* Open the "Colab Notebooks" folder
* Read the file containing the key (you may need to change the file name in the with open command below)
* Set the OPENAI_API_KEY environmental *variable*

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

with open("/content/drive/My Drive/Colab Notebooks/openai") as f:
  OPENAI_token = f.read().strip()

import os
os.environ["OPENAI_API_KEY"]=OPENAI_token

Mounted at /content/drive


* You can also just paste the key replacing "your_openai_api_key_here" in the cell below but that's not a good practice if you share your notebook with someone else!

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"

### Prerequisites

Before we start building our chatbot, we need to install some Python libraries. You can install these libraries using pip as shown below:


In [2]:
!pip install -qU \
    langchain==0.2.1 \
    langchain-community==0.2.1 \
    langchain-openai==0.1.8 \
    openai==1.30.5 \
    faiss-gpu==1.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 16.6 MB/s eta 0:00:00


### STEP 1: Let's build a chatbot without RAG first.

We will be relying heavily on the LangChain library to bring together the different components needed for our chatbot. To begin, we'll create a simple chatbot without any retrieval augmentation. We do this by initializing a `ChatOpenAI` object. You need an [OpenAI API key](https://platform.openai.com/account/api-keys) to accomplish this step.

In [3]:
import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

chat = ChatOpenAI(
    model='gpt-4'
)

Next, we need to introduce [LangChain](https://python.langchain.com/v0.2/docs/introduction/). LangChain is an open source orchestration framework for the development of applications using large language models (LLMs). Its tools and APIs simplify the process of building LLM-driven applications like chatbots and virtual agents.

In LangChain, we use three _message_ objects to construct a conversation as shown below:

In [5]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="How do I return my LED TV?")
]

Then, we generate the next response from the AI by passing these messages to the `ChatOpenAI` object.

In [6]:
res = chat.invoke(messages)
res

AIMessage(content="Returning your LED TV may depend on the store's return policy where you bought it, but here's a general process:\n\n1. Check Return Policy: Ensure that the store where you purchased the LED TV allows returns. Some stores have a specific time period during which returns can be made, typically around 30 days. \n\n2. Keep Your Receipt: Always keep your purchase receipt. This can be either a paper receipt or an online confirmation of your order. \n\n3. Original Packaging: If possible, return the TV in its original packaging. This includes all manuals, accessories, and any free gifts that came with the TV.\n\n4. Contact Customer Service: Contact the store's customer service department. They will provide specific instructions on how to return your TV. If you purchased the TV online, they might email you a return shipping label.\n\n5. Shipping: If you're shipping the TV, make sure it's packed securely to prevent any damage during transport. You may prefer to use a shipping 

In response we get another AI message object. We can print it more clearly like so:

In [7]:
print(res.content)

Returning your LED TV may depend on the store's return policy where you bought it, but here's a general process:

1. Check Return Policy: Ensure that the store where you purchased the LED TV allows returns. Some stores have a specific time period during which returns can be made, typically around 30 days. 

2. Keep Your Receipt: Always keep your purchase receipt. This can be either a paper receipt or an online confirmation of your order. 

3. Original Packaging: If possible, return the TV in its original packaging. This includes all manuals, accessories, and any free gifts that came with the TV.

4. Contact Customer Service: Contact the store's customer service department. They will provide specific instructions on how to return your TV. If you purchased the TV online, they might email you a return shipping label.

5. Shipping: If you're shipping the TV, make sure it's packed securely to prevent any damage during transport. You may prefer to use a shipping company that offers insurance

Because `res` is just another `AIMessage` object, we can append it to `messages`, add another `HumanMessage`, and generate the next response in the conversation.

In [8]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="How long would it take for the refund to reflect in my bank account?"
)
# add to messages
messages.append(prompt)

# send to ChatOpenAI
res = chat.invoke(messages)

print(res.content)

The time it takes for a refund to appear in your bank account can vary depending on the retailer and the bank. Generally, it can take anywhere from 3-5 business days up to 7-10 business days. However, some banks or credit card companies may take up to 30 days to process a refund. If you don't see the refund in your account after this time, it's a good idea to follow up with both the retailer and your bank.


### STEP 2: Now we have a basic chatbot. Let's work on building our vector knowledge base.


We have our chatbot, but notice that it is giving generic responses. The reason for this is that the LLM is using general knowledge about returning products. It has no knowledge about our return policies and procedures.

LLMs learn all they know during training. An LLM essentially compresses the "world" as seen in the training data into the internal parameters of the model.  We call this knowledge the _parametric knowledge_ of the model.

By default, LLMs have no access to the external world or private databases. We provide additional, targeted, information to the LLM using vector databases.

So first, we'll need a dataset. I've provided you with a synthetic e-commerce customer service dataset below and I'll walk you through the steps to set up the vector store. First let's take a look at how to generate vector embeddings.

Step 2.1: **Generate Vector Embeddings**

- Prepare Your Data: Gather the text data you want to include in your knowledge base. In our case, the synthetic customer service data.
- Generate Embeddings: Use a pre-trained model (like OpenAI's `text-embedding-ada-002` model or other embedding models) to convert your text data into vector embeddings. Each text entry will be represented as a vector of numbers.


In [9]:
from langchain_openai import OpenAIEmbeddings

openai_api_key = os.environ.get('OPENAI_API_KEY')
model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=openai_api_key
)

# Sample FAQ data
faq_data = [
    "To return your LED TV, ensure it is in its original packaging and includes all accessories. Returns are accepted within 30 days of purchase. A restocking fee of 15% may apply.",
    "Laptops can be returned within 14 days of purchase. Please make sure the device is in its original condition with all accessories and packaging. A restocking fee of 10% may apply.",
    "For Camera returns, please ensure the camera, lens, and all accessories are included in the original packaging. Returns are accepted within 30 days, and items must be in new condition.",
    "Home appliances can be returned within 30 days of purchase. Make sure the appliance is unused, in its original packaging, and includes all accessories. A restocking fee of 10% may apply.",
    "You can track your order by logging into your account, going to 'My Orders', and clicking on 'Track Order' next to the relevant order.",
    "We offer standard and express shipping options. Standard shipping typically takes 5-7 business days, while express shipping takes 2-3 business days.",
    "For any questions or assistance, please contact our customer support team via the 'Contact Us' page on our website, by phone at 1-800-123-4567, or by email at support@example.com.",
    "To navigate our e-commerce site, use the menu at the top of the page to browse different categories. You can use the search bar to find specific items. For detailed product information, click on the product image or title.",
    "Shipping costs are calculated at checkout based on your location and the shipping method selected. We offer free shipping on orders over $50.",
    "To change or cancel your order, please contact our customer support team as soon as possible. If the order has not yet been processed, we will be able to make the changes or cancel it for you.",
    "Payments can be declined for various reasons, such as incorrect card details, insufficient funds, or issues with the card issuer. Please check your details and try again or contact your bank for more information.",
    "If your order has not yet been shipped, you can change the shipping address by contacting our customer support team.",
    "If you receive a damaged or defective item, please contact our customer support team immediately. We will arrange for a replacement or a refund."
    "Refund to store credit typically happens instantly. Refund to your original form of purchase such as credit/debit card, bank account, etc might take 7-14 business days to process."
]



Step 2.2: **Setup Vectorstore using FAISS(Facebook AI Similarity Search)**
- In the cell below, we demonstrate how to use the faiss package to create a FAISS Index directly from colab
- FAISS (Facebook AI Similarity Search) is a python library for searching and clustering vectors. Recall that we use embedded word vectors to store knowledge in text AI applications. FAISS, with its similarity functionality, helps us retreive knowledge effectively
- For more information on Facebook AI Similarity Search, see: https://ai.meta.com/tools/faiss/

In [10]:
from langchain_community.vectorstores import FAISS
import faiss

# Add embeddings to the FAISS index
faiss_index = FAISS.from_texts(faq_data, embed)

### STEP 3: We are ready to connect the chatbot to our vector knowledge base using RAG.

We've built a fully-fledged knowledge base. Now it's time to connect that knowledge base to our chatbot. To do that, I've prepared the retrive_similar_faq function that based on the query, retrives the top 3 similar customer service question from the database we provided earlier.

In [11]:
import numpy as np

def retrieve_similar_faq(query):
    query_embedding = embed.embed_query(query)
    query_embedding_np = np.array(query_embedding).astype("float32").reshape(1, -1)  # Convert to numpy array
    D, I = faiss_index.index.search(query_embedding_np, k=3)
    return [faq_data[i] for i in I[0]]

Let's see how it works.

In [12]:
query = "How do I return my LED TV?"
similar_faqs = retrieve_similar_faq(query)
for faq in similar_faqs:
    print(faq)

To return your LED TV, ensure it is in its original packaging and includes all accessories. Returns are accepted within 30 days of purchase. A restocking fee of 15% may apply.
Home appliances can be returned within 30 days of purchase. Make sure the appliance is unused, in its original packaging, and includes all accessories. A restocking fee of 10% may apply.
Laptops can be returned within 14 days of purchase. Please make sure the device is in its original condition with all accessories and packaging. A restocking fee of 10% may apply.


We return a lot of text here and it's not that clear what we need or what is relevant. Fortunately, our LLM will be able to parse this information much faster than us. All we need is to connect the output from our `retrieve_similar_faq` to our `chat` chatbot. To do that we can use the same logic as we used earlier.

In [13]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = retrieve_similar_faq(query)
    # get the text from the results
    source_knowledge = "\n".join(results)
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

Using this we produce an augmented prompt:

In [14]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    To return your LED TV, ensure it is in its original packaging and includes all accessories. Returns are accepted within 30 days of purchase. A restocking fee of 15% may apply.
Home appliances can be returned within 30 days of purchase. Make sure the appliance is unused, in its original packaging, and includes all accessories. A restocking fee of 10% may apply.
Laptops can be returned within 14 days of purchase. Please make sure the device is in its original condition with all accessories and packaging. A restocking fee of 10% may apply.

    Query: How do I return my LED TV?


There is still a lot of text here, so let's pass it onto our chat model to see how it performs.

### Result:

We get a much more informed response that includes several items from our knowldge base.

In [15]:
messages_RAG = [
    SystemMessage(content="You are a helpful assistant.")
]

query = "How do I return my LED TV?"
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages_RAG.append(prompt)

res_RAG = chat.invoke(messages_RAG)

print(res_RAG.content)

To return your LED TV, make sure it is in its original packaging and includes all accessories. Returns for LED TVs are accepted within 30 days of purchase. Please note that a restocking fee of 15% may apply.


In [16]:
# add latest AI response to messages
messages_RAG.append(res_RAG)

query = "How long would it take for the refund to reflect in my bank account?"
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)

# add to messages
messages_RAG.append(prompt)

# send to ChatOpenAI
res_RAG = chat.invoke(messages_RAG)

print(res_RAG.content)

The refund to your original form of purchase such as a bank account might take 7-14 business days to process.


## Compare the Vanilla GPT4 with RAG GPT4
---
## Question: How do I return my LED TV?

### Vanilla Answer:

Returning an LED TV generally depends on the store from where it was purchased. However, here are some general steps that you can follow:

1. **Check the Store's Return Policy**: This will typically be found on their website or on your receipt.

2. **Pack the TV**: If you have the original packaging, use that to repack the TV securely. If not, you'll need to find a box that will fit the TV and add enough padding to prevent any damage.

3. **Contact the Store**: Reach out to the store's customer service and let them know you want to return the TV. They'll guide you through their specific process. This might involve providing a receipt or proof of purchase.

4. **Ship or Drop Off**: Depending on the store's policy, you might need to ship the TV back to them or drop it off at a store location.

Remember, every store has different policies, and some might not accept returns after a certain period of time or if the TV isn't in its original condition. Always check the specifics with the store where you purchased the TV.

### RAG Answer:

To return your LED TV, ensure that the TV is in its original packaging and includes all the accessories that it came with. The return must be made within 30 days of the purchase. Please note, a restocking fee of 15% may apply.

---

## Next question: How long would it take for the refund to reflect in my bank account

### Vanilla Answer:
The time it takes for a refund to appear in your bank account can vary widely based on the retailer and your bank. Generally, it can take anywhere from a few business days up to 10 business days. Some banks may take even longer to process the refund transaction.

Once the retailer processes your refund, they will typically provide some sort of confirmation. If you don't see the refund in your account after a reasonable amount of time, it might be a good idea to contact both the retailer and your bank to check on the status.


### RAG Answer:

The refund to your original form of purchase such as a bank account might take 7-14 business days to process.

---

# What if we ask RAG Chatbot something it doesn't know?

How about we try asking the vanilla GPT4 and our RAG Chatbot a question not in the faq such as "How should I return the water bottle?"

## vanilla GPT4

In [17]:
# vanilla GPT4
query = "How should I return the water bottle?"
# create a new user prompt
prompt = HumanMessage(
    content=query
)

# add to messages
messages.append(prompt)

# send to ChatOpenAI
res = chat.invoke(messages)

print(res.content)

Returning your water bottle is similar to returning other items. Here's a general guideline:

1. Check Return Policy: Look at the store's or website's return policy where you purchased the water bottle. Some places have a time limit for returns, often around 30 days.

2. Receipt: Keep your receipt or order confirmation as proof of purchase.

3. Condition: The water bottle should be in good condition. If it's used, you may need to clean it before returning. If it's defective, this should be specified when returning.

4. Contact Customer Service: Reach out to the store's customer service department for specific instructions on how to return your water bottle. If you purchased it online, they might provide a return shipping label.

5. Packaging: Pack the water bottle securely to prevent any damage during transport if you're shipping it back. 

6. Return: Follow the store's instructions to return the item. This may involve shipping it back or returning it to a physical store location.

7. 

## RAG

In [18]:
# RAG
query = "How should I return the water bottle?"
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)

# add to messages
messages_RAG.append(prompt)

# send to ChatOpenAI
res_RAG = chat.invoke(messages_RAG)

print(res_RAG.content)

The contexts provided do not contain specific information on how to return a water bottle. Please refer to the appropriate return policy or contact customer service for further assistance.


In [19]:
query = "How should I return the water bottle?"
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    Home appliances can be returned within 30 days of purchase. Make sure the appliance is unused, in its original packaging, and includes all accessories. A restocking fee of 10% may apply.
For Camera returns, please ensure the camera, lens, and all accessories are included in the original packaging. Returns are accepted within 30 days, and items must be in new condition.
Laptops can be returned within 14 days of purchase. Please make sure the device is in its original condition with all accessories and packaging. A restocking fee of 10% may apply.

    Query: How should I return the water bottle?


## Compare the results:
---
## Question: How do I return my LED TV?
### Vanilla :
Returning a water bottle depends on the store from where it was purchased and their return policy. Here are some general steps that you can follow:

1. **Check the Store's Return Policy**: Every store's return policy is different. Some may allow returns within a certain period of time, while others might not accept returns at all. This information should be available on the store's website or on your receipt.

2. **Prepare the water bottle for Return**: If possible, clean and dry the water bottle. Repackage it in its original packaging if you have it.

3. **Contact the Store**: Reach out to the store's customer service and inform them that you want to return the water bottle. They can provide you with specific instructions.

4. **Ship or Drop Off**: Depending on the store's policy, you might need to ship the water bottle back to them or drop it off at a store location.

Remember to always check the specifics with the store where you purchased the water bottle.

### RAG:
I'm sorry, but the contexts provided do not contain information on how to return a water bottle. Please refer to the specific return policy for water bottles or contact customer service.

---

## Comments:
Upon reviewing the augemented prompt, we see the below prompt:

### Prompt:

Using the contexts below, answer the query.

Contexts:

Home appliances can be returned within 30 days of purchase. Make sure the appliance is unused, in its original packaging, and includes all accessories. A restocking fee of 10% may apply.

For Camera returns, please ensure the camera, lens, and all accessories are included in the original packaging. Returns are accepted within 30 days, and items must be in new condition.

Laptops can be returned within 14 days of purchase. Please make sure the device is in its original condition with all accessories and packaging. A restocking fee of 10% may apply.

Query:

How should I return the water bottle?"

### Analysis:
Due to lack of relevant context, our RAG chatbot explicitly states that the provided contexts do not include specific information on how to return a water bottle and advises referring to the specific return policy or contacting customer service. This approach avoids giving potentially incorrect or irrelevant information by sticking closely to the provided contexts.

---

---

In [22]:
# RAG
query = "You don't know the process for returning a water bottle but could you outline a general return process for me?"
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)

# add to messages
messages_RAG.append(prompt)

# send to ChatOpenAI
res_RAG = chat.invoke(messages_RAG)

print(res_RAG.content)

Sure, while the specific process may vary slightly depending on the product, here's a general overview of a return process based on the provided contexts:

1. Make sure your product is within the return period. For instance, home appliances and cameras have a 30-day return period, while laptops have a 14-day return period.

2. Ensure that the product is in its original condition. It should be unused and should include all accessories.

3. Pack the product in its original packaging.

4. Be aware that a restocking fee may apply. This fee is typically 10%, but for certain items like LED TVs, it can be 15%.

5. Once you have prepared the item for return, contact the customer service team to initiate the return process. 

Please note that these steps are general and may not apply to all items or situations. Always check the specific return policy for the item you purchased.
